<img src = "https://upload.wikimedia.org/wikipedia/commons/3/32/Earth_rotation.gif" align = right width = 150 height = 450><a id=1></a>
<h2> Mapping in Python</h2>
<h1 align = left> A project in heatmapping NYC arrest data using Python...</h1>
<h3> Amelia Ingram </h3><br>

This project was inspired by a couple [sources](#7):  first, a GeoPython course by Henrikki Tenkanen and George Pipis' blog post on creating heatmaps.  Just for a frame of reference, I just finished my first Python course and won't be able to take data visualization until next year, but wanted to attempt to learn some mapping in the meanwhile with some NYC open data.  (By the way, this is also my first attempt with going "off road" out of a guided course).  The following is the result of my exploration of this technique... </p>  

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install streamlit-folium #leaflet visualization

     |████████████████████████████████| 9.7 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 19.1 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 164 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 807 kB/s eta 0:00:01
     |████████████████████████████████| 111 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 812 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 180 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 961 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 1.7 MB/s eta 0:00:01


     |████████████████████████████████| 339 kB 3.4 MB/s eta 0:00:01
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13451 sha256=83aa0ec92b90fa45108e919b2ccc6b2e3332e6c96955727960d562fa1caf56ee
  Stored in directory: /Users/ameliaingram/Library/Caches/pip/wheels/b7/a5/68/fe632054a5eadd531c7a49d740c50eb6adfbeca822b4eab8d4
Successfully built blinker


In [4]:
import streamlit as st
from streamlit_folium import folium_static
import folium


In [5]:
# Create a Map instance
m = folium.Map(location=[40.6973,-74.1515], tiles='Stamen Toner',
                   zoom_start=10, control_scale=True)
print(m)

In [6]:
#Save the base map output
outfp = "base_map.html"

In [2]: m.save(outfp)

In [8]:
# Let's change the basemap style to 'Stamen Toner'
m2 = folium.Map(location=[40.6973,-74.1515], tiles='Stamen Toner',
                zoom_start=12, control_scale=True, prefer_canvas=True)

 # Filepath to the output
outfp2 = "/Users/ameliaingram/Documents/My_GitHub+Repository/base_map2.html"

# Save the map
m2.save(outfp2)

In [6]:
#Install Geopandas which will help us to use layers on the base map
import sys
!{sys.executable} -m pip install geopandas #map layers visualization

In [10]:
import geopandas as gp
from folium.plugins import HeatMap
import pandas as pd
import numpy as np

In [11]:
# load the data
arrest_table = pd.read_csv("NYPD_Arrest_Data__Year_to_Date_.csv")

In [12]:
print(arrest_table.head())

   ARREST_KEY ARREST_DATE  PD_CD                   PD_DESC  KY_CD  \
0   234233843  09/29/2021  105.0         STRANGULATION 1ST  106.0   
1   234129823  09/27/2021  157.0                    RAPE 1  104.0   
2   234040747  09/25/2021  109.0  ASSAULT 2,1,UNCLASSIFIED  106.0   
3   234047720  09/25/2021  101.0                 ASSAULT 3  344.0   
4   234042526  09/25/2021  101.0                 ASSAULT 3  344.0   

                      OFNS_DESC    LAW_CODE LAW_CAT_CD ARREST_BORO  \
0                FELONY ASSAULT  PL 1211200          F           B   
1                          RAPE  PL 1303501          F           K   
2                FELONY ASSAULT  PL 1200501          F           Q   
3  ASSAULT 3 & RELATED OFFENSES  PL 1200001          M           B   
4  ASSAULT 3 & RELATED OFFENSES  PL 1200001          M           B   

   ARREST_PRECINCT  JURISDICTION_CODE AGE_GROUP PERP_SEX PERP_RACE  \
0               42                  0     25-44        M     BLACK   
1               77      

In [13]:
#pull locations from the table to determine the starting geopoint
locations = arrest_table[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

[40.81038342800008, -73.90452841699994]

In [14]:
# start the map of New York using the latitude and longitude from locations
map_osm = folium.Map(location=[40.81, -73.90], zoom_start=11)
map_osm

In [18]:
#Now we will work with the Race and take a sample of 1K observations.
arrest_table = arrest_table.sample(n=1000, replace=False, random_state=1)
arrest_table.dropna(subset=['PERP_RACE'],inplace=True)

#We will create a function to produce a heatmap based on race
def race_col(x):
  if x=='BLACK':
    return 'black'
  if x=='WHITE':
    return 'blue'
  if x=='ASIAN':
    return 'yellow'
  if x=='HISPANIC':
    return 'red'
  else:
    return 'green'
arrest_table['color_race'] = arrest_table['PERP_RACE'].apply(lambda x:race_col(x) )

# Create the Heat Map
#Make the list of Lat an Long
lat = arrest_table.Latitude.tolist()
lng = arrest_table.Longitude.tolist()
HeatMap(list(zip(lat, lng))).add_to(map_osm)

# Add the Circles with the corresponding races based on the color
arrest_table.apply(lambda x:folium.Circle(location=[x['Latitude'], x['Longitude']], radius=50, fill=True, color=x['color_race'], popup=x['AGE_GROUP']).add_to(map_osm), axis=1)
map_osm

<h3>Success!</h3>  
<p>The result generated a heatmap of a 1000 person sample of arrests, sorted by race.  The individual points are arrests that are color coded by race, and will pop up with age.  </p>
<p>I still have a lot to learn on this technique (and mapping in general)...but I made some progress and it was a lot of fun!</p>  


<h3>Sources Used in this Project:</h3><a id=7></a>
NYC Open Data.  <a>https://opendata.cityofnewyork.us/</a><br>
Pipis, George.  2021.  How to Make Interactive Maps with Folium.  Python-bloggers.com <a>https://python-bloggers.com/2021/06/how-to-make-interactive-maps-with-folium/ </a><br>
Silva, George.  2017.  Mapping Points with Folium.  <a>https://georgetsilva.github.io/posts/mapping-points-with-folium/</a><br>
Tenkanen, Henrikki.  2016. GeoPython and ArcGIS. Fall 2016 course materials.  University of Helsinki.  <a>https://automating-gis-processes.github.io/2016/course-info.html#</a> <br>

[Back to top](#1)